<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/mandarin_frequency_analysis/pinyin_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using this to provide example word for the heisig deck

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url_start = "https://learn-chinese-words.com/dictionary.php?input="
char = "弟"
char = "吾"
source = requests.get(url_start+char).text
soup = BeautifulSoup(source, 'html.parser')
results = soup.find('table', {'class':'results'})
results

<table class="results">
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/吾">吾</a></td><td>wu2 </td><td>I</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/支吾">支吾</a></td><td>zhi1 wu2 </td><td>to elude</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/吾人">吾人</a></td><td>wu2 ren2 </td><td></td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/维吾尔">维吾尔</a></td><td>wei2 wu2 er3 </td><td>Uighur ethnic group of Xinjiang</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/维吾尔族">维吾尔族</a></td><td>wei2 wu2 er3 zu2 </td><td>Uighur (Uyghur) ethnic group of Xinjiang</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/伊吾">伊吾</a></td><td>yi1 wu2 </td><td>Yiwu county in Kumul prefecture 哈密地區哈密地区[Ha1 mi4 di4 qu1], Xinjiang</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/支吾其辞">支吾其辞</a></td><td>zhi1 wu2 qi2 ci2 </td><td>prevaricate</td></tr>
<tr><td><a h

In [3]:
df = pd.read_html(str(results))[0]
df = df.fillna('')
df

,0,1,2
0,吾,wu2,I
1,支吾,zhi1 wu2,to elude
2,吾人,wu2 ren2,
3,维吾尔,wei2 wu2 er3,Uighur ethnic group of Xinjiang
4,维吾尔族,wei2 wu2 er3 zu2,Uighur (Uyghur) ethnic group of Xinjiang
5,伊吾,yi1 wu2,Yiwu county in Kumul prefecture 哈密地區哈密地区[Ha1 m...
6,支吾其辞,zhi1 wu2 qi2 ci2,prevaricate
7,维吾尔人,wei2 wu2 er3 ren2,Uighur person or people
8,老吾老,lao3 wu2 lao3,
9,张吾乐,zhang1 wu2 le4,Zhang Wule


In [4]:
# lifted from here https://stackoverflow.com/questions/8200349/convert-numbered-pinyin-to-pinyin-with-tone-marks
import re

pinyinToneMarks = {
    'a': 'āáǎà', 'e': 'ēéěè', 'i': 'īíǐì',
    'o': 'ōóǒò', 'u': 'ūúǔù', 'ü': 'ǖǘǚǜ',
    'A': 'ĀÁǍÀ', 'E': 'ĒÉĚÈ', 'I': 'ĪÍǏÌ',
    'O': 'ŌÓǑÒ', 'U': 'ŪÚǓÙ', 'Ü': 'ǕǗǙǛ'
}

def convertPinyinCallback(m):
    tone=int(m.group(3))%5
    r=m.group(1).replace('v', 'ü').replace('V', 'Ü')
    # for multple vowels, use first one if it is a/e/o, otherwise use second one
    pos=0
    if len(r)>1 and not r[0] in 'aeoAEO':
        pos=1
    if tone != 0:
        r=r[0:pos]+pinyinToneMarks[r[pos]][tone-1]+r[pos+1:]
    return r+m.group(2)

def convertPinyin(s):
    return re.sub(r'([aeiouüvÜ]{1,3})(n?g?r?)([012345])', convertPinyinCallback, s, flags=re.IGNORECASE)

print(convertPinyin('Ni3 hao3 ma0?'))

Nǐ hǎo ma?


In [5]:
df[1] = df[1].apply(convertPinyin)
df[2] = df[2].apply(convertPinyin)
df

,0,1,2
0,吾,wú,I
1,支吾,zhī wú,to elude
2,吾人,wú rén,
3,维吾尔,wéi wú ěr,Uighur ethnic group of Xinjiang
4,维吾尔族,wéi wú ěr zú,Uighur (Uyghur) ethnic group of Xinjiang
5,伊吾,yī wú,Yiwu county in Kumul prefecture 哈密地區哈密地区[Hā mì...
6,支吾其辞,zhī wú qí cí,prevaricate
7,维吾尔人,wéi wú ěr rén,Uighur person or people
8,老吾老,lǎo wú lǎo,
9,张吾乐,zhāng wú lè,Zhang Wule


In [6]:
ch = list(df[0])
py = list(df[1])
en = list(df[2])

samples = '\n'.join(['-'.join(sample) for sample in list(zip(ch,py,en))])
samples

'吾-wú-I\n支吾-zhī wú-to elude\n吾人-wú rén-\n维吾尔-wéi wú ěr-Uighur ethnic group of Xinjiang\n维吾尔族-wéi wú ěr zú-Uighur (Uyghur) ethnic group of Xinjiang\n伊吾-yī wú-Yiwu county in Kumul prefecture 哈密地區哈密地区[Hā mì dì qū], Xinjiang\n支吾其辞-zhī wú qí cí-prevaricate\n维吾尔人-wéi wú ěr rén-Uighur person or people\n老吾老-lǎo wú lǎo-\n张吾乐-zhāng wú lè-Zhang Wule\n新疆维吾尔自治区-xīn jiāng wéi wú ěr zì zhì qū-Xinjiang Uighur Autonomous Region\n支支吾吾-zhī zhī wú wú-to stall\n吾尔开希-wú ěr kāi xī-rkesh Dlet (1968-), one of the main leaders of the Beijing student democracy movement of 1989'

In [7]:
print(samples)

吾-wú-I
支吾-zhī wú-to elude
吾人-wú rén-
维吾尔-wéi wú ěr-Uighur ethnic group of Xinjiang
维吾尔族-wéi wú ěr zú-Uighur (Uyghur) ethnic group of Xinjiang
伊吾-yī wú-Yiwu county in Kumul prefecture 哈密地區哈密地区[Hā mì dì qū], Xinjiang
支吾其辞-zhī wú qí cí-prevaricate
维吾尔人-wéi wú ěr rén-Uighur person or people
老吾老-lǎo wú lǎo-
张吾乐-zhāng wú lè-Zhang Wule
新疆维吾尔自治区-xīn jiāng wéi wú ěr zì zhì qū-Xinjiang Uighur Autonomous Region
支支吾吾-zhī zhī wú wú-to stall
吾尔开希-wú ěr kāi xī-rkesh Dlet (1968-), one of the main leaders of the Beijing student democracy movement of 1989


In [8]:
file_to_edit = "https://raw.githubusercontent.com/Tclack88/misc-analysis/main/mandarin_frequency_analysis/Heisig1_to_edit.csv"
headers =  ["Hanzi", "Meaning", "Notes", "Words"]
old = pd.read_csv(file_to_edit, index_col=False, names=headers)
old.head()

,Hanzi,Meaning,Notes,Words
0,一,one,yī,NaN
1,二,two,èr,NaN
2,三,three,sān,NaN
3,四,four,sì,NaN
4,五,five,wǔ,NaN


In [50]:
test_slice = old[88:108]
test_slice

,Hanzi,Meaning,Notes,Words
88,孔,"cavity, orifice, keyhole",kǒng<br><br>King KONG's baby slipped through t...,NaN
89,吼,roar,hǒu<br><br>We only found out (king kong's baby...,NaN
90,了,(-ed) (makes past tense),le<br>,NaN
91,女,woman,nǚ,NaN
92,好,good,hǎo,NaN
93,如,"be like, comply","rú<br><br>Remember from 如果 (if)<br><br>Also, b...",NaN
94,母,mother,mǔ<br><br>Pictograph. Depicts breasts,NaN
95,贯,pierce,guàn<br><br>Everyone in GUANtanamo Bay had pie...,NaN
96,兄,elder brother,xiōng<br><br>I used to speak (think of word sh...,NaN
97,克,overcome,"""kè<br><br>I COUld overcome 10 older brothers ...",NaN


time: 20.5 ms (started: 2021-12-02 05:43:04 +00:00)


In [10]:
def get_example_words(char):
  try:
    url_start = "https://learn-chinese-words.com/dictionary.php?input="
    source = requests.get(url_start+char).text
    soup = BeautifulSoup(source, 'html.parser')
    results = soup.find('table', {'class':'results'})
    df = pd.read_html(str(results))[0].fillna('') #str for convertPinyin, NaN=>''
    df[1] = df[1].apply(convertPinyin)
    df[2] = df[2].apply(convertPinyin)
    ch = list(df[0])
    py = list(df[1])
    en = list(df[2])
    samples = '<br>'.join(['-'.join(sample) for sample in list(zip(ch,py,en))])
    return samples
  except Exception as e:
    print(f'problem withi {char}:\n{e}')
    return ''


# test_slice['Words'] = test_slice['Hanzi'].apply(get_example_words)

In [51]:
test_slice

,Hanzi,Meaning,Notes,Words
88,孔,"cavity, orifice, keyhole",kǒng<br><br>King KONG's baby slipped through t...,NaN
89,吼,roar,hǒu<br><br>We only found out (king kong's baby...,NaN
90,了,(-ed) (makes past tense),le<br>,NaN
91,女,woman,nǚ,NaN
92,好,good,hǎo,NaN
93,如,"be like, comply","rú<br><br>Remember from 如果 (if)<br><br>Also, b...",NaN
94,母,mother,mǔ<br><br>Pictograph. Depicts breasts,NaN
95,贯,pierce,guàn<br><br>Everyone in GUANtanamo Bay had pie...,NaN
96,兄,elder brother,xiōng<br><br>I used to speak (think of word sh...,NaN
97,克,overcome,"""kè<br><br>I COUld overcome 10 older brothers ...",NaN


time: 18.6 ms (started: 2021-12-02 05:43:10 +00:00)


Test done. Time for actual conversion

In [12]:
!pip install ipython-autotime

%load_ext autotime

time: 226 µs (started: 2021-12-02 05:04:25 +00:00)


In [13]:
old.head(5)

,Hanzi,Meaning,Notes,Words
0,一,one,yī,NaN
1,二,two,èr,NaN
2,三,three,sān,NaN
3,四,four,sì,NaN
4,五,five,wǔ,NaN


time: 18.2 ms (started: 2021-12-02 05:04:29 +00:00)


In [ ]:
old['Words'] = old['Hanzi'].apply(get_example_words)

Improved hunt with more common words

In [56]:
test_slice = old.sample(n=25)
for ch in test_slice.Hanzi:
  print(ch)
  #I noticed the search bar populates with the hex code of the character
  #ex. 四 -> ox56db -> /5/56DB (the first number is repeated)
  tail = str(hex(ord(ch))).upper()[2:] 
  url = f'http://chinese-characters.org/contained/{tail[0]}/{tail}.html'
  print(url)
  try:
    a = pd.read_html(url, header=0)
  except Exception as e:
    print(e)
    print(f'{ch} NOT FOUND')
    continue
  print(len(a))
  good_tables = []
  for table in a:
    clean_table = table.dropna(axis=0,how='all').dropna(axis=1, how='all')
    if clean_table.columns.to_list() == ['Simp', 'Pinyin', 'English']:
      good_tables.append(clean_table)
  print(len(good_tables))
  #last = a[-1].dropna(axis=0,how='all').dropna(axis=1, how='all')
  #twotolast = a[-2].dropna(axis=0,how='all').dropna(axis=1, how='all')
  #print(last.columns)
  #print(last.head(3))
  #print(twotolast.columns)
  #print(twotolast.head(3))

价
http://chinese-characters.org/contained/4/4EF7.html
5
0
堡
http://chinese-characters.org/contained/5/5821.html
7
2
无
http://chinese-characters.org/contained/6/65E0.html
5
0
讨
http://chinese-characters.org/contained/8/8BA8.html
HTTP Error 404: Not Found
讨 NOT FOUND
板
http://chinese-characters.org/contained/6/677F.html
7
2
迈
http://chinese-characters.org/contained/8/8FC8.html
HTTP Error 404: Not Found
迈 NOT FOUND
器
http://chinese-characters.org/contained/5/5668.html
7
2
故
http://chinese-characters.org/contained/6/6545.html
8
2
温
http://chinese-characters.org/contained/6/6E29.html
6
0
袭
http://chinese-characters.org/contained/8/88AD.html
HTTP Error 404: Not Found
袭 NOT FOUND
沼
http://chinese-characters.org/contained/6/6CBC.html
8
2
区
http://chinese-characters.org/contained/5/533A.html
HTTP Error 404: Not Found
区 NOT FOUND
革
http://chinese-characters.org/contained/9/9769.html
10
2
颗
http://chinese-characters.org/contained/9/9897.html
HTTP Error 404: Not Found
颗 NOT FOUND
激
http://chinese-

In [38]:
a = pd.read_html('http://chinese-characters.org/contained/5/53E3.html', header=0)
a

[                                                  Simp  ... Words
 0                                                  NaN  ...   NaN
 1                                                    𧥣  ...     0
 2                                                    𢒫  ...     0
 3                                                    𨙫  ...     0
 4                                                    害  ...    13
 ..                                                 ...  ...   ...
 689                                                长江口  ...   NaN
 690                                                喷火口  ...   NaN
 691                                                注入口  ...   NaN
 692                                                NaN  ...   NaN
 693  Home  Pinyin index  Radical index  Links  Abou...  ...   NaN
 
 [694 rows x 15 columns], Empty DataFrame
 Columns: [Home  Pinyin index  Radical index  Links  About  Search for a character:  Please spread the word!  var addthis_config = {data_track_addressba

time: 2.87 s (started: 2021-12-02 05:29:04 +00:00)


In [43]:
a[7].dropna(axis=0,how='all').dropna(axis=1, how='all')

,Simp,Trad,Pinyin,English,Semantic,Phonetic,Words
1,吡,吡,BI3,blame,NaN,NaN,2.0
2,呋,呋,FU1,NaN,NaN,NaN,1.0
3,咣,咣,GUANG1,NaN,NaN,NaN,0.0
4,啉,啉,LAN2,slow,NaN,NaN,2.0
5,嘭,嘭,PENG1,drive_away,NaN,NaN,0.0
...,...,...,...,...,...,...,...
558,哫,哫,zu2,cajole,NaN,NaN,0.0
559,嘴,嘴,zui3,mouth,NaN,NaN,13.0
560,噿,噿,zui3,NaN,NaN,NaN,0.0
561,噂,噂,zun3,meet,口,尊,0.0


time: 34.6 ms (started: 2021-12-02 05:30:10 +00:00)
